In [46]:
from clipper_admin import ClipperConnection, DockerContainerManager
clipper_conn = ClipperConnection(DockerContainerManager())
clipper_conn.start_clipper()

20-01-13:01:08:18 INFO     [docker_container_manager.py:184] [default-cluster] Starting managed Redis instance in Docker
20-01-13:01:08:22 INFO     [docker_container_manager.py:276] [default-cluster] Metric Configuration Saved at /tmp/tmp2p0j480q.yml
20-01-13:01:08:24 INFO     [clipper_admin.py:162] [default-cluster] Clipper is running


In [47]:
clipper_conn.register_application(name="squares", input_type="doubles", default_output="-1.0", slo_micros=100000)

20-01-13:01:08:29 INFO     [clipper_admin.py:236] [default-cluster] Application squares was successfully registered


In [48]:
clipper_conn.get_all_apps()

['squares']

In [49]:
#求位平方和函数
def square(n):
    n = str(n) #先把数字转化为字符串
    sum = 0
    for i in n:     
        sum += int(i) ** 2 #计算字符串中每个数的平方
    return str(sum)

In [50]:
from clipper_admin.deployers import python as python_deployer

In [51]:
python_deployer.deploy_python_closure(clipper_conn, name="sum-square", version=1, input_type="doubles", func=square)

20-01-13:01:08:54 INFO     [deployer_utils.py:41] Saving function to /tmp/tmpunkku2zdclipper
20-01-13:01:08:54 INFO     [deployer_utils.py:51] Serialized and supplied predict function
20-01-13:01:08:54 INFO     [python.py:192] Python closure saved
20-01-13:01:08:54 INFO     [python.py:206] Using Python 3.6 base image
20-01-13:01:08:54 INFO     [clipper_admin.py:534] [default-cluster] Building model Docker image with model data from /tmp/tmpunkku2zdclipper
20-01-13:01:08:55 INFO     [clipper_admin.py:539] [default-cluster] Step 1/2 : FROM clipper/python36-closure-container:0.4.1
20-01-13:01:08:55 INFO     [clipper_admin.py:539] [default-cluster]  ---> e5b9dc250c05
20-01-13:01:08:55 INFO     [clipper_admin.py:539] [default-cluster] Step 2/2 : COPY /tmp/tmpunkku2zdclipper /model/
20-01-13:01:08:55 INFO     [clipper_admin.py:539] [default-cluster]  ---> 1d85085640cf
20-01-13:01:08:55 INFO     [clipper_admin.py:539] [default-cluster] Successfully built 1d85085640cf
20-01-13:01:08:55 INFO   

In [52]:
clipper_conn.link_model_to_app(app_name="squares", model_name="sum-square")

20-01-13:01:09:01 INFO     [clipper_admin.py:303] [default-cluster] Model sum-square is now linked to application squares


In [63]:
import requests, json, numpy as np
headers = {"Content-type": "application/json"}
requests.post("http://localhost:1337/squares/predict", headers=headers, data=json.dumps({"input": [123.]})).json()

{'query_id': 2,
 'output': -1.0,
 'default': True,
 'default_explanation': 'No connected models found for query'}

In [64]:
clipper_conn.get_all_apps()

['squares']

In [65]:
clipper_conn.stop_all()

20-01-13:01:55:39 INFO     [clipper_admin.py:1424] [default-cluster] Stopped all Clipper cluster and all model containers


In [ ]:
1,定义函数 参数 必须是 列表
2，该函数应该接受input_type指定的类型的输入列表，并将预测的Python列表或numpy数组作为字符串返回。
3,定义请求名字 单词间不能有空格